# Collecting Geo Data for Seattle

In [1]:
#importing the libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import re #regular expessions

! pip install beautifulsoup4
! pip install requests
import requests #request online data
from bs4 import BeautifulSoup as bs # library for navigating html files

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
#load the seattle_neighborhoods data
seattle_df = pd.read_csv('seattle_data.csv')
seattle_df.head()

,Within larger district,Neighborhood name
0,Seattle,North Seattle
1,North Seattle,Broadview
2,North Seattle,Bitter Lake
3,North Seattle,North Beach
4,North Seattle,Crown Hill


In [2]:
CLIENT_ID = '4TJ1SLTDO1KADL553GXCDXOJXZONZYTBIRQMVBQ32JYAU3EA' # your Foursquare ID
CLIENT_SECRET = 'TC1N45PW5LSGVLIZ02DDPA2JOMWB2I3BA33OPX55VW2LBKBF' # your Foursquare Secret
ACCESS_TOKEN = 'NS3WWURVAWWAGWSTUXQDQCS0150IPNQIYNHVE5JM1OSX2CAV' # your FourSquare Access Token

In [1]:
VERSION = '20180604'
LIMIT = 30

In [4]:
hood_longitude_list = list()
hood_latitude_list = list()
for neighborhood in seattle_df['Neighborhood name']:
    try:
        address = '{}, Seattle, WA'.format(neighborhood)
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        hood_longitude_list.append(longitude)
        hood_latitude_list.append(latitude)
    except:
        latitude = 'No Data Found'
        longitude = 'No Data Found'
        hood_longitude_list.append(longitude)
        hood_latitude_list.append(latitude)
print('hoods printed')

hoods printed


In [5]:
area_longitude_list = list()
area_latitude_list = list()
for area in seattle_df['Within larger district']:
    try:
        address = '{}, Seattle, WA'.format(area)
        geolocator = Nominatim(user_agent="foursquare_agent")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        area_longitude_list.append(longitude)
        area_latitude_list.append(latitude)
    except:
        latitude = 'No Data Found'
        longitude = 'No Data Found'
        area_longitude_list.append(longitude)
        area_latitude_list.append(latitude)
print('areas printed')

areas printed


In [6]:
seattle_df.insert(loc=2,
    column='area_longitude',
    value=area_longitude_list
)
seattle_df.insert(loc=3,
    column='area_latitude',
    value=area_latitude_list
)
seattle_df.insert(loc=4,
    column='hood_longitude',
    value=hood_longitude_list
)
seattle_df.insert(loc=5,
    column='hood_latitude',
    value=hood_latitude_list
)
seattle_df.head()

,Within larger district,Neighborhood name,area_longitude,area_latitude,hood_longitude,hood_latitude
0,Seattle,North Seattle,-122.34,47.602,-122.291,47.5901
1,North Seattle,Broadview,-122.291,47.5901,-122.36,47.7223
2,North Seattle,Bitter Lake,-122.291,47.5901,-122.349,47.7262
3,North Seattle,North Beach,-122.291,47.5901,-122.392,47.6962
4,North Seattle,Crown Hill,-122.291,47.5901,-122.371,47.6947


In [7]:
#to prove df is storing correct values and not rounding them
for arealat in seattle_df['area_latitude'][0:5]:
    print(arealat)

47.6020234
47.590054550000005
47.590054550000005
47.590054550000005
47.590054550000005


In [61]:
seattle_df = seattle_df.set_index('Within larger district')

In [62]:
seattle_df.to_csv('seattle_geodata.csv')

## Visualizing Neighborhoods

In [63]:
print(location)

Delridge, Seattle, King County, Washington, 98106-1499, United States


In [9]:
#folium.map cannot have string values for latitude and longitude
for lat,lng in zip(hood_latitude_list, hood_longitude_list):
    if type(lat)== float or type(lat)==int:
        latitude=lat
        longitude=lng
        
#map each neighborhood and area
map_seattle = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, neighborhood in zip(seattle_df['hood_latitude'], seattle_df['hood_longitude'], seattle_df['Neighborhood name']):
    try:
        label = '{}'.format(neighborhood)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_seattle)
    except:
        continue
for lat, lng, area in zip(seattle_df['area_latitude'], seattle_df['area_longitude'],seattle_df['Within larger district']):
    try:
        label = '{}'.format(area)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=1.0,
            parse_html=False).add_to(map_seattle)
    except:
        continue
map_seattle